In [86]:
import pandas as pd
import scanpy as sc
import sys
import numpy as np
import scipy
import joblib

In [87]:
out_file_path = './'
data_dir = out_file_path + 'data/'
gene_signature_dir = out_file_path + 'gene_signatures/'

From DESeq2, look for enrichment of metabolism terms in DE genes

**read gene signatures**

In [88]:
mouse_filename = gene_signature_dir + 'metabolism_terms_mouse.pickle'
mouse_terms = joblib.load(mouse_filename)

In [89]:
for k,v in mouse_terms.items():
    print(f'{k}: {len(v)}')

KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS: 66
KEGG_MM_CITRATE_CYCLE: 33
KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY: 30
KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS: 34
KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM: 40
KEGG_MM_GALACTOSE_METABOLISM: 31
KEGG_MM_ASCORBATE_AND_ALDARATE_METABOLISM: 27
KEGG_MM_FATTY_ACID_BIOSYNTHESIS: 6
KEGG_MM_FATTY_ACID_METABOLISM: 48
KEGG_MM_SYNTHESIS_AND_DEGRADATION_OF_KETONE_BODIES: 11
KEGG_MM_STEROID_BIOSYNTHESIS: 17
KEGG_MM_PRIMARY_BILE_ACID_BIOSYNTHESIS: 16
KEGG_MM_STEROID_HORMONE_BIOSYNTHESIS: 58
KEGG_MM_OXIDATIVE_PHOSPHORYLATION: 142
KEGG_MM_PURINE_METABOLISM: 170
KEGG_MM_CAFFEINE_METABOLISM: 9
KEGG_MM_PYRIMIDINE_METABOLISM: 100
KEGG_MM_ALANINE_ASPARTATE_AND_GLUTAMATE_METABOLISM: 35
KEGG_MM_GLYCINE_SERINE_AND_THREONINE_METABOLISM: 39
KEGG_MM_CYSTEINE_AND_METHIONINE_METABOLISM: 39
KEGG_MM_VALINE_LEUCINE_AND_ISOLEUCINE_DEGRADATION: 53
KEGG_MM_VALINE_LEUCINE_AND_ISOLEUCINE_BIOSYNTHESIS: 2
KEGG_MM_LYSINE_BIOSYNTHESIS: 5
KEGG_MM_LYSINE_DEGRADATION: 48
KEGG_MM_ARGININE_AND

In [118]:
[i for i in mouse_terms.keys() if 'BILE' in i]

['KEGG_MM_PRIMARY_BILE_ACID_BIOSYNTHESIS']

In [90]:
# read in DE genes
compass_data = pd.read_csv(data_dir + 'th_data/GSE162300_DFMO_RNA_est_counts.csv', sep = ',', index_col = 0) # this is library size normalized already
samples = list(set([i[:-9] for i in compass_data.columns]))

In [91]:
n_genes_total = len(compass_data.index)
n_genes_total

20817

In [92]:
samples

['Th17p_DFMO',
 'Th17n_Vehicle',
 'Th17n_DFMO',
 'iTreg_Vehicle',
 'Th17p_Vehicle',
 'iTreg_DFMO']

In [93]:
de_genes = {}
i =0 
for s1 in samples:
    for s2 in samples:
        if s1 != s2:
            de_genes[str(i)] = {'s1': s1, 's2': s2, 'res': pd.read_csv(data_dir + 'th_data/deseq_genes_%s_%s.csv' %(s1, s2), index_col = 0)}
            de_genes[str(i)]['res']['genes'] = de_genes[str(i)]['res'].index
            i+=1

In [130]:
# explore de_genes
de_genes['21']['res']

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,genes
0610007P14RIK,877.903660,-0.138205,0.058002,-2.382782,0.017182,0.045688,0610007P14RIK
0610009B22RIK,108.511004,0.248351,0.119890,2.071492,0.038313,0.089526,0610009B22RIK
0610009L18RIK,3.271914,-0.947371,0.680663,-1.391835,0.163972,0.283326,0610009L18RIK
0610009O20RIK,203.816852,-0.099596,0.098636,-1.009738,0.312621,0.456212,0610009O20RIK
0610010F05RIK,28.814349,-0.088189,0.229593,-0.384111,0.700896,0.797399,0610010F05RIK
...,...,...,...,...,...,...,...
ZYG11A,0.000000,NaN,NaN,NaN,NaN,NaN,ZYG11A
ZYG11B,270.947472,0.093160,0.088846,1.048551,0.294385,0.436705,ZYG11B
ZYX,370.487493,-0.450275,0.099703,-4.516177,0.000006,0.000041,ZYX
ZZEF1,234.860096,0.156738,0.099569,1.574168,0.115449,0.215660,ZZEF1


In [131]:
genes_look_for = [('CMPK1', 74),
 ('HEXA', 23),
 ('HEXB', 23),
 ('GM20390', 23),
 ('NME2', 23),
 ('CTSA', 22),
 ('GALNS', 22),
 ('GLB1', 22),
 ('NEU1', 22),
 ('EHHADH', 20),
 ('FPGS', 18),
 ('CPT1A', 16),
 ('B4GALT1', 16),
 ('B4GALT2', 16),
 ('B4GALT3', 16),
 ('B4GALT5', 16),
 ('B3GNT2', 15),
 ('B3GNT3', 15),
 ('B3GNT4', 15),
 ('B3GNT7', 15),
 ('B3GNT8', 15),
 ('B3GNTL1', 15),
 ('B4GAT1', 15),
 ('ACADM', 14),
 ('GNS', 14),
 ('CPT1B', 13),
 ('CPT1C', 13),
 ('ACADS', 13),
 ('MAN1A', 13),
 ('MAN1A2', 13),
 ('MAN1C1', 13),
 ('ACAA1B', 12),
 ('PNP2', 12),
 ('ACSL1', 11),
 ('FUT9', 11),
 ('HSD17B4', 10)]

In [133]:
genes_look_for = [i[0] for i in genes_look_for]

In [143]:
len(genes_look_for)

36

In [144]:
genes_look_for = [i for i in genes_look_for if i in compass_data.index]

In [145]:
len(genes_look_for)

34

In [220]:
de_genes['21']['res'][de_genes['21']['res']['padj'] < 0.05].sort_values('log2FoldChange')

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,genes
GJA5,17.785087,-6.910474,0.873197,-7.913992,2.492650e-15,6.061075e-14,GJA5
LAMC3,12.139044,-6.836304,0.881275,-7.757284,8.676748e-15,1.993649e-13,LAMC3
HAVCR1,14.813155,-6.320211,0.867933,-7.281914,3.291166e-13,6.392572e-12,HAVCR1
IL9,2075.828929,-6.053138,0.204654,-29.577491,2.910706e-192,7.530578e-189,IL9
PELI2,12.929091,-5.823323,0.886516,-6.568772,5.073185e-11,7.398728e-10,PELI2
...,...,...,...,...,...,...,...
SLC17A6,46.107814,6.399281,0.715206,8.947462,3.637502e-19,1.321762e-17,SLC17A6
IFITM1,9.252409,6.425785,0.972825,6.605284,3.967550e-11,5.879063e-10,IFITM1
MX1,11.583791,6.638496,0.905678,7.329861,2.303908e-13,4.585131e-12,MX1
DCN,4.103664,6.801086,2.320935,2.930322,3.386112e-03,1.145469e-02,DCN


In [221]:
t = de_genes['21']['res'].loc[genes_look_for].copy()

In [222]:
t[(t['padj'] < 0.05)].sort_values('log2FoldChange')

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,genes
B3GNT8,16.609205,-2.522312,0.404549,-6.234867,4.521622e-10,5.757057e-09,B3GNT8
B4GALT1,2193.156958,-1.207372,0.075424,-16.007747,1.128226e-57,3.040570e-55,B4GALT1
GLB1,385.376957,-0.590349,0.105522,-5.594538,2.212100e-08,2.199518e-07,GLB1
GALNS,114.133926,-0.417528,0.118859,-3.512805,4.434026e-04,1.909406e-03,GALNS
CPT1A,189.398484,-0.345988,0.104969,-3.296112,9.803295e-04,3.841727e-03,CPT1A
B4GALT3,319.903669,-0.287474,0.080814,-3.557234,3.747808e-04,1.646349e-03,B4GALT3
HEXB,293.860724,-0.270958,0.105574,-2.566510,1.027276e-02,2.980230e-02,HEXB
HSD17B4,379.325281,0.181457,0.071585,2.534856,1.124935e-02,3.209564e-02,HSD17B4
FPGS,214.720402,0.265079,0.100741,2.631280,8.506387e-03,2.539161e-02,FPGS
NEU1,282.920847,0.300784,0.071963,4.179704,2.918892e-05,1.667055e-04,NEU1


In [223]:
len(t[(t['padj'] < 0.05)].sort_values('log2FoldChange'))

15

In [165]:
# these are for thP vs N

In [161]:
de_genes_th_1 = pd.read_csv(data_dir + 'th_data/deseq_genes_th_uncollapse.csv', index_col = 0)
de_genes_th_2 = pd.read_csv(data_dir + 'th_data/deseq_genes_th_collapse.csv', index_col = 0)

In [171]:
t_1 = de_genes_th_1.loc[genes_look_for].copy()
t_2 = de_genes_th_2.loc[genes_look_for].copy()

In [176]:
len(genes_look_for)

34

In [177]:
len(t_1[t_1['padj'] < 0.05].sort_values('log2FoldChange'))

16

In [224]:
len(t_2[t_2['padj'] < 0.05].sort_values('log2FoldChange'))

6

In [174]:
t_1[t_1['padj'] < 0.05].sort_values('log2FoldChange')

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
B3GNT8,30.570997,-2.510049,0.276799,-9.068127,1.210807e-19,2.534423e-18
B4GALT1,1035.367292,-1.223586,0.081249,-15.059671,2.982671e-51,2.394844e-49
GLB1,364.518023,-0.589519,0.092623,-6.364726,1.956383e-10,2.066190e-09
GALNS,101.459089,-0.427471,0.121046,-3.531479,4.132421e-04,1.726561e-03
CPT1A,181.843657,-0.350504,0.105247,-3.330302,8.675181e-04,3.342595e-03
B4GALT3,289.807188,-0.298178,0.086456,-3.448893,5.628901e-04,2.274611e-03
HEXB,365.452178,-0.273308,0.078628,-3.475982,5.089869e-04,2.083494e-03
HSD17B4,352.279093,0.175330,0.067878,2.582999,9.794573e-03,2.814905e-02
FPGS,174.744035,0.257054,0.104724,2.454586,1.410468e-02,3.854782e-02
NEU1,252.509783,0.294949,0.078793,3.743325,1.816007e-04,8.213970e-04


In [175]:
t_2[t_2['padj'] < 0.05].sort_values('log2FoldChange')

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
B3GNT8,61.372016,-2.516839,0.451241,-5.577597,2.438641e-08,6.247193e-07
B4GALT1,2076.861845,-1.229532,0.118886,-10.342102,4.544601e-25,6.121935e-23
GLB1,730.510972,-0.595894,0.149750,-3.979264,6.912898e-05,8.380053e-04
CMPK1,1143.854239,0.380509,0.110414,3.446204,5.685205e-04,5.263314e-03
B3GNT2,756.440266,0.646698,0.141796,4.560765,5.096770e-06,8.324956e-05
B4GALT5,81.953190,1.128046,0.366308,3.079498,2.073495e-03,1.559544e-02


# hypergeometric test

In [94]:
from utils import hypergeometric_test, adjust_p_value_fdr

In [95]:
logfc_thresh = 1
p_thresh = 0.01

In [100]:
M = n_genes_total
hypergeom_res = {k: {} for k in de_genes.keys()}
for samp in de_genes.keys():
    df = de_genes[samp]['res']
    df = df[df['log2FoldChange'] > logfc_thresh]
    df = df[df['padj'] < p_thresh]
    df['genes'] = df['genes'].str.upper()
    df_genes = set(df['genes'])
    N = len(df) # length of DE genes
    for k,v in mouse_terms.items():
        n = len(v) # number of met genes
        x = len(df_genes.intersection(v))
        pct_overlap = x/(N+n)
        
        if pct_overlap > 0:
            p_val = hypergeometric_test(total_genes_expressed=M, n_genes_of_interest=n, 
                                        n_genes_picked=N, n_overlap=x)
            hypergeom_res[samp][k] =  p_val
        else:
            hypergeom_res[samp][k] = 1

In [101]:
hypergeom_df = pd.DataFrame.from_dict(hypergeom_res)
hypergeom_df['term'] = hypergeom_df.index
hypergeom_df = hypergeom_df.melt('term')
hypergeom_df['p'] = hypergeom_df['value']
hypergeom_df['padj'] = adjust_p_value_fdr(hypergeom_df['p'])
# add sample info
hypergeom_df['s1'] = None
hypergeom_df['s2'] = None
for k,v in de_genes.items():
    hypergeom_df.loc[hypergeom_df['variable'] == k, 's1'] = v['s1']
    hypergeom_df.loc[hypergeom_df['variable'] == k, 's2'] = v['s2']
hypergeom_df

,term,variable,value,p,padj,s1,s2
0,KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS,0,1.000000,1.000000,1.000000,Th17p_DFMO,Th17n_Vehicle
1,KEGG_MM_CITRATE_CYCLE,0,1.000000,1.000000,1.000000,Th17p_DFMO,Th17n_Vehicle
2,KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY,0,1.000000,1.000000,1.000000,Th17p_DFMO,Th17n_Vehicle
3,KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS,0,0.695560,0.695560,0.869034,Th17p_DFMO,Th17n_Vehicle
4,KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM,0,0.753245,0.753245,0.939307,Th17p_DFMO,Th17n_Vehicle
...,...,...,...,...,...,...,...
2605,KEGG_MM_MTOR_SIGNALING_PATHWAY,29,0.102240,0.102240,0.163710,iTreg_DFMO,Th17p_Vehicle
2606,KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,29,0.079875,0.079875,0.127976,iTreg_DFMO,Th17p_Vehicle
2607,KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,29,0.422855,0.422855,0.677918,iTreg_DFMO,Th17p_Vehicle
2608,KEGG_MM_INSULIN_SIGNALING_PATHWAY,29,0.816993,0.816993,1.000000,iTreg_DFMO,Th17p_Vehicle


In [102]:
hypergeom_df[hypergeom_df['padj'] < 0.05].value_counts('variable')

variable
9     21
24    15
5     13
4     12
6     11
22    11
19    10
7      8
11     7
14     6
20     6
23     6
26     6
15     5
17     5
12     4
29     4
27     4
25     4
13     4
0      4
1      3
28     3
21     3
16     3
8      3
3      2
2      2
18     2
10     2
dtype: int64

In [103]:
hypergeom_df.to_csv(out_file_path + 'compass_th_hypergeom_results.csv')

In [105]:
hypergeom_paper = hypergeom_df[hypergeom_df['variable'] == '21']
hypergeom_paper[hypergeom_paper['padj'] < 0.05].to_csv(out_file_path + 'compass_th_hypergeom_results_th17_control.csv')

In [104]:
hypergeom_df[hypergeom_df['padj'] < 0.05]

,term,variable,value,p,padj,s1,s2
38,KEGG_MM_N-GLYCAN_BIOSYNTHESIS,0,0.001513,0.001513,0.009140,Th17p_DFMO,Th17n_Vehicle
46,KEGG_MM_GLYCOSAMINOGLYCAN_BIOSYNTHESIS_KERATAN...,0,0.013483,0.013483,0.040449,Th17p_DFMO,Th17n_Vehicle
70,KEGG_MM_FOLATE_BIOSYNTHESIS,0,0.008918,0.008918,0.023371,Th17p_DFMO,Th17n_Vehicle
83,KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,0,0.011471,0.011471,0.017653,Th17p_DFMO,Th17n_Vehicle
118,KEGG_MM_TAURINE_AND_HYPOTAURINE_METABOLISM,1,0.026969,0.026969,0.047690,Th17p_DFMO,Th17n_DFMO
...,...,...,...,...,...,...,...
2519,KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,28,0.021172,0.021172,0.038861,iTreg_DFMO,iTreg_Vehicle
2569,KEGG_MM_GLYCOSAMINOGLYCAN_BIOSYNTHESIS_KERATAN...,29,0.020330,0.020330,0.030814,iTreg_DFMO,Th17p_Vehicle
2579,KEGG_MM_GLYCOSPHINGOLIPID_BIOSYNTHESIS_LACTO_N...,29,0.016541,0.016541,0.026966,iTreg_DFMO,Th17p_Vehicle
2581,KEGG_MM_GLYCOSPHINGOLIPID_BIOSYNTHESIS_GANGLIO...,29,0.020330,0.020330,0.032917,iTreg_DFMO,Th17p_Vehicle


In [106]:
hypergeom_paper[hypergeom_paper['padj'] < 0.05]

,term,variable,value,p,padj,s1,s2
1883,KEGG_MM_GLYCOSPHINGOLIPID_BIOSYNTHESIS_LACTO_N...,21,0.017385,0.017385,0.019507,Th17p_Vehicle,Th17n_Vehicle
1910,KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,21,0.006110,0.006110,0.006576,Th17p_Vehicle,Th17n_Vehicle
1911,KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,21,0.001929,0.001929,0.002121,Th17p_Vehicle,Th17n_Vehicle


In [80]:
for k,v in de_genes.items():
    print(f"{k}: {v['s1']}, {v['s2']}")

0: Th17p_DFMO, Th17n_Vehicle
1: Th17p_DFMO, Th17n_DFMO
2: Th17p_DFMO, iTreg_Vehicle
3: Th17p_DFMO, Th17p_Vehicle
4: Th17p_DFMO, iTreg_DFMO
5: Th17n_Vehicle, Th17p_DFMO
6: Th17n_Vehicle, Th17n_DFMO
7: Th17n_Vehicle, iTreg_Vehicle
8: Th17n_Vehicle, Th17p_Vehicle
9: Th17n_Vehicle, iTreg_DFMO
10: Th17n_DFMO, Th17p_DFMO
11: Th17n_DFMO, Th17n_Vehicle
12: Th17n_DFMO, iTreg_Vehicle
13: Th17n_DFMO, Th17p_Vehicle
14: Th17n_DFMO, iTreg_DFMO
15: iTreg_Vehicle, Th17p_DFMO
16: iTreg_Vehicle, Th17n_Vehicle
17: iTreg_Vehicle, Th17n_DFMO
18: iTreg_Vehicle, Th17p_Vehicle
19: iTreg_Vehicle, iTreg_DFMO
20: Th17p_Vehicle, Th17p_DFMO
21: Th17p_Vehicle, Th17n_Vehicle
22: Th17p_Vehicle, Th17n_DFMO
23: Th17p_Vehicle, iTreg_Vehicle
24: Th17p_Vehicle, iTreg_DFMO
25: iTreg_DFMO, Th17p_DFMO
26: iTreg_DFMO, Th17n_Vehicle
27: iTreg_DFMO, Th17n_DFMO
28: iTreg_DFMO, iTreg_Vehicle
29: iTreg_DFMO, Th17p_Vehicle


In [64]:
hypergeom_res

{'0': {'KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS': 0,
  'KEGG_MM_CITRATE_CYCLE': 0,
  'KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY': 0,
  'KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS': 0.6955601115895058,
  'KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM': 0.7532452550186388,
  'KEGG_MM_GALACTOSE_METABOLISM': 0,
  'KEGG_MM_ASCORBATE_AND_ALDARATE_METABOLISM': 0,
  'KEGG_MM_FATTY_ACID_BIOSYNTHESIS': 0,
  'KEGG_MM_FATTY_ACID_METABOLISM': 0.8135442679435141,
  'KEGG_MM_SYNTHESIS_AND_DEGRADATION_OF_KETONE_BODIES': 0,
  'KEGG_MM_STEROID_BIOSYNTHESIS': 0,
  'KEGG_MM_PRIMARY_BILE_ACID_BIOSYNTHESIS': 0,
  'KEGG_MM_STEROID_HORMONE_BIOSYNTHESIS': 0.8686601784523652,
  'KEGG_MM_OXIDATIVE_PHOSPHORYLATION': 0.9581689594881504,
  'KEGG_MM_PURINE_METABOLISM': 0.23075233629023514,
  'KEGG_MM_CAFFEINE_METABOLISM': 0,
  'KEGG_MM_PYRIMIDINE_METABOLISM': 0.12983892860597496,
  'KEGG_MM_ALANINE_ASPARTATE_AND_GLUTAMATE_METABOLISM': 0.7060337929739788,
  'KEGG_MM_GLYCINE_SERINE_AND_THREONINE_METABOLISM': 0.744451473297975,
  '

In [124]:
hypergeom_paper[hypergeom_paper['padj'] < 0.05]

,term,variable,value,p,padj,s1,s2
1883,KEGG_MM_GLYCOSPHINGOLIPID_BIOSYNTHESIS_LACTO_N...,21,0.017385,0.017385,0.019507,Th17p_Vehicle,Th17n_Vehicle
1910,KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,21,0.006110,0.006110,0.006576,Th17p_Vehicle,Th17n_Vehicle
1911,KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,21,0.001929,0.001929,0.002121,Th17p_Vehicle,Th17n_Vehicle


In [127]:
hypergeom_paper[hypergeom_paper['padj'] < 0.05].iloc[0]['term']

'KEGG_MM_GLYCOSPHINGOLIPID_BIOSYNTHESIS_LACTO_NEOLACTO_SERIES'

In [126]:
paper_pathways = pd.read_csv('./metabolism_pathways.txt', sep = ' \n', header = None)
paper_pathways

/tmp/ipykernel_2650081/2502518715.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  paper_pathways = pd.read_csv('./metabolism_pathways.txt', sep = ' \n', header = None)


,0
0,ROS detoxification
1,Heparan sulfate degradation
2,Chondroitin sulfate degradation
3,Pyrimidine catabolism
4,Brood group synthesis
5,Purine synthesis
6,Methionine and cysteine metabolism
7,Pyrimidine synthesis
8,Sphingolipid metabolism
9,Purine catabolism


# less stringent thresholds for hypergeom

In [200]:
logfc_thresh = 0
p_thresh = 0.05

In [214]:
M = n_genes_total
hypergeom_res_relax = {k: {} for k in de_genes.keys()}
for samp in de_genes.keys():
    df = de_genes[samp]['res']
    df = df[df['log2FoldChange'] > logfc_thresh]
    df = df[df['padj'] < p_thresh]
    df['genes'] = df['genes'].str.upper()
    df_genes = set(df['genes'])
    print(f'{samp}: {len(df)}')
    N = len(df) # length of DE genes
    for k,v in mouse_terms.items():
        n = len(v) # number of met genes
        x = len(df_genes.intersection(v))
        pct_overlap = x/(N+n)
        
        if pct_overlap > 0:
            p_val = hypergeometric_test(total_genes_expressed=M, n_genes_of_interest=n, 
                                        n_genes_picked=N, n_overlap=x)
            hypergeom_res_relax[samp][k] =  p_val
        else:
            hypergeom_res_relax[samp][k] = 1

0: 3603
1: 2225
2: 3233
3: 2920
4: 3186
5: 3910
6: 3945
7: 3250
8: 2370
9: 4301
10: 2172
11: 3500
12: 3203
13: 3386
14: 2711
15: 3461
16: 3378
17: 3537
18: 2935
19: 3630
20: 3457
21: 2570
22: 3976
23: 3082
24: 4266
25: 3153
26: 4011
27: 2707
28: 3024
29: 3757


In [213]:
M = n_genes_total
hypergeom_res_relax_flip = {k: {} for k in de_genes.keys()}
for samp in de_genes.keys():
    df = de_genes[samp]['res']
    df = df[df['log2FoldChange'] < logfc_thresh]
    df = df[df['padj'] < p_thresh]
    print(f'{samp}: {len(df)}')
    df['genes'] = df['genes'].str.upper()
    df_genes = set(df['genes'])
    N = len(df) # length of DE genes
    for k,v in mouse_terms.items():
        n = len(v) # number of met genes
        x = len(df_genes.intersection(v))
        pct_overlap = x/(N+n)
        
        if pct_overlap > 0:
            p_val = hypergeometric_test(total_genes_expressed=M, n_genes_of_interest=n, 
                                        n_genes_picked=N, n_overlap=x)
            hypergeom_res_relax_flip[samp][k] =  p_val
        else:
            hypergeom_res_relax_flip[samp][k] = 1

0: 3910
1: 2172
2: 3461
3: 3457
4: 3153
5: 3603
6: 3500
7: 3378
8: 2570
9: 4011
10: 2225
11: 3945
12: 3537
13: 3976
14: 2707
15: 3233
16: 3250
17: 3203
18: 3082
19: 3024
20: 2920
21: 2370
22: 3386
23: 2935
24: 3757
25: 3186
26: 4301
27: 2711
28: 3630
29: 4266


In [203]:
hypergeom_df_relax = pd.DataFrame.from_dict(hypergeom_res_relax)
hypergeom_df_relax['term'] = hypergeom_df_relax.index
hypergeom_df_relax = hypergeom_df_relax.melt('term')
hypergeom_df_relax['p'] = hypergeom_df_relax['value']
hypergeom_df_relax['padj'] = adjust_p_value_fdr(hypergeom_df_relax['p'])
# add sample info
hypergeom_df_relax['s1'] = None
hypergeom_df_relax['s2'] = None

In [204]:
hypergeom_df_relax_flip = pd.DataFrame.from_dict(hypergeom_res_relax_flip)
hypergeom_df_relax_flip['term'] = hypergeom_df_relax_flip.index
hypergeom_df_relax_flip = hypergeom_df_relax_flip.melt('term')
hypergeom_df_relax_flip['p'] = hypergeom_df_relax_flip['value']
hypergeom_df_relax_flip['padj'] = adjust_p_value_fdr(hypergeom_df_relax_flip['p'])
# add sample info
hypergeom_df_relax_flip['s1'] = None
hypergeom_df_relax_flip['s2'] = None

In [209]:
for k,v in de_genes.items():
    hypergeom_df_relax.loc[hypergeom_df_relax['variable'] == k, 's1'] = v['s1']
    hypergeom_df_relax.loc[hypergeom_df_relax['variable'] == k, 's2'] = v['s2']
    hypergeom_df_relax_flip.loc[hypergeom_df_relax_flip['variable'] == k, 's1'] = v['s1']
    hypergeom_df_relax_flip.loc[hypergeom_df_relax_flip['variable'] == k, 's2'] = v['s2']
hypergeom_df_relax

,term,variable,value,p,padj,s1,s2
0,KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS,0,0.904529,0.904529,0.963600,Th17p_DFMO,Th17n_Vehicle
1,KEGG_MM_CITRATE_CYCLE,0,0.019251,0.019251,0.022047,Th17p_DFMO,Th17n_Vehicle
2,KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY,0,0.612011,0.612011,0.635128,Th17p_DFMO,Th17n_Vehicle
3,KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS,0,0.949056,0.949056,1.000000,Th17p_DFMO,Th17n_Vehicle
4,KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM,0,0.845634,0.845634,0.899757,Th17p_DFMO,Th17n_Vehicle
...,...,...,...,...,...,...,...
2605,KEGG_MM_MTOR_SIGNALING_PATHWAY,29,0.000034,0.000034,0.000079,iTreg_DFMO,Th17p_Vehicle
2606,KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,29,0.121250,0.121250,0.132912,iTreg_DFMO,Th17p_Vehicle
2607,KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,29,0.639608,0.639608,0.700242,iTreg_DFMO,Th17p_Vehicle
2608,KEGG_MM_INSULIN_SIGNALING_PATHWAY,29,0.004942,0.004942,0.005475,iTreg_DFMO,Th17p_Vehicle


In [206]:
hypergeom_df_relax[hypergeom_df_relax['padj'] < 0.05].value_counts('variable')

variable
14    28
9     27
19    27
17    25
22    25
6     23
7     22
4     22
24    21
5     20
20    20
23    20
8     17
15    13
29    13
27    13
12    12
13    12
25    11
1     11
21    11
3     11
11    11
26    10
28    10
2     10
10    10
18    10
16    10
0      9
dtype: int64

In [207]:
hypergeom_paper_relax = hypergeom_df_relax[hypergeom_df_relax['variable'] == '21']
hypergeom_paper_relax[hypergeom_paper_relax['padj'] < 0.05]

,term,variable,value,p,padj,s1,s2
1841,KEGG_MM_PURINE_METABOLISM,21,3.828712e-04,3.828712e-04,0.001085,Th17p_Vehicle,Th17n_Vehicle
1843,KEGG_MM_PYRIMIDINE_METABOLISM,21,2.003687e-07,2.003687e-07,0.000001,Th17p_Vehicle,Th17n_Vehicle
1845,KEGG_MM_GLYCINE_SERINE_AND_THREONINE_METABOLISM,21,1.755234e-02,1.755234e-02,0.018775,Th17p_Vehicle,Th17n_Vehicle
1846,KEGG_MM_CYSTEINE_AND_METHIONINE_METABOLISM,21,5.397952e-04,5.397952e-04,0.002243,Th17p_Vehicle,Th17n_Vehicle
1848,KEGG_MM_VALINE_LEUCINE_AND_ISOLEUCINE_BIOSYNTH...,21,1.523638e-02,1.523638e-02,0.031890,Th17p_Vehicle,Th17n_Vehicle
1860,KEGG_MM_CYANOAMINO_ACID_METABOLISM,21,2.826717e-03,2.826717e-03,0.003401,Th17p_Vehicle,Th17n_Vehicle
1863,KEGG_MM_GLUTATHIONE_METABOLISM,21,3.364438e-02,3.364438e-02,0.035609,Th17p_Vehicle,Th17n_Vehicle
1865,KEGG_MM_N-GLYCAN_BIOSYNTHESIS,21,7.388616e-04,7.388616e-04,0.033249,Th17p_Vehicle,Th17n_Vehicle
1887,KEGG_MM_GLYOXYLATE_AND_DICARBOXYLATE_METABOLISM,21,1.069665e-02,1.069665e-02,0.012661,Th17p_Vehicle,Th17n_Vehicle
1905,KEGG_MM_METABOLIC_PATHWAYS,21,3.044201e-06,3.044201e-06,0.000005,Th17p_Vehicle,Th17n_Vehicle


In [210]:
hypergeom_paper_relax_flip = hypergeom_df_relax_flip[hypergeom_df_relax_flip['variable'] == '21']
hypergeom_paper_relax_flip[hypergeom_paper_relax_flip['padj'] < 0.05]

,term,variable,value,p,padj,s1,s2
1831,KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM,21,1.229432e-02,1.229432e-02,1.673875e-02,Th17p_Vehicle,Th17n_Vehicle
1832,KEGG_MM_GALACTOSE_METABOLISM,21,6.174934e-03,6.174934e-03,2.940981e-02,Th17p_Vehicle,Th17n_Vehicle
1835,KEGG_MM_FATTY_ACID_METABOLISM,21,2.165119e-03,2.165119e-03,4.239280e-03,Th17p_Vehicle,Th17n_Vehicle
1836,KEGG_MM_SYNTHESIS_AND_DEGRADATION_OF_KETONE_BO...,21,4.869631e-03,4.869631e-03,9.393745e-03,Th17p_Vehicle,Th17n_Vehicle
1837,KEGG_MM_STEROID_BIOSYNTHESIS,21,3.652292e-02,3.652292e-02,4.580722e-02,Th17p_Vehicle,Th17n_Vehicle
1850,KEGG_MM_LYSINE_DEGRADATION,21,1.708593e-02,1.708593e-02,1.964506e-02,Th17p_Vehicle,Th17n_Vehicle
1869,KEGG_MM_AMINO_SUGAR_AND_NUCLEOTIDE_SUGAR_METAB...,21,8.366369e-04,8.366369e-04,1.423483e-03,Th17p_Vehicle,Th17n_Vehicle
1871,KEGG_MM_GLYCOSAMINOGLYCAN_DEGRADATION,21,2.597535e-02,2.597535e-02,4.247849e-02,Th17p_Vehicle,Th17n_Vehicle
1873,KEGG_MM_GLYCOSAMINOGLYCAN_BIOSYNTHESIS_KERATAN...,21,2.147261e-02,2.147261e-02,2.391955e-02,Th17p_Vehicle,Th17n_Vehicle
1876,KEGG_MM_INOSITOL_PHOSPHATE_METABOLISM,21,2.901964e-03,2.901964e-03,6.435111e-03,Th17p_Vehicle,Th17n_Vehicle


In [211]:
print(len(hypergeom_paper_relax[hypergeom_paper_relax['padj'] < 0.05]))

11


In [212]:
print(len(hypergeom_paper_relax_flip[hypergeom_paper_relax_flip['padj'] < 0.05]))

16


# for DE genes of Th17p vs Th17n

In [179]:
logfc_thresh = 0
p_thresh = 0.05

In [180]:
de_genes_th_1

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0610007P14RIK,944.099532,-0.145766,0.050880,-2.864867,4.171846e-03,1.344874e-02
0610009B22RIK,86.186921,0.244914,0.142373,1.720221,8.539226e-02,1.719417e-01
0610009L18RIK,3.371014,-0.814473,0.686218,-1.186901,2.352666e-01,3.753754e-01
0610009O20RIK,195.335549,-0.110608,0.093428,-1.183884,2.364589e-01,3.767663e-01
0610010F05RIK,25.537423,-0.075539,0.247169,-0.305619,7.598948e-01,8.430450e-01
...,...,...,...,...,...,...
ZYG11A,0.000000,NaN,NaN,NaN,NaN,NaN
ZYG11B,288.482467,0.080973,0.090098,0.898727,3.687982e-01,5.198402e-01
ZYX,398.640411,-0.455847,0.070043,-6.508119,7.609765e-11,8.329504e-10
ZZEF1,212.350099,0.151869,0.107047,1.418710,1.559836e-01,2.755397e-01


In [199]:
M = n_genes_total
hypergeom_res_th_1 = {k: {} for k in ['pvn', 'nvp']}
for key in ['pvn', 'nvp']:
    df = de_genes_th_1.copy()
    df = df[df['padj'] < p_thresh]
    if key == 'pvn':
        df = df[df['log2FoldChange'] > logfc_thresh]
    else:
        df = df[df['log2FoldChange'] < logfc_thresh]
    print(f'{key}: {len(df)}')
    df['genes'] = df.index.str.upper()
    df_genes = set(df['genes'])
    N = len(df) # length of DE genes
    for k,v in mouse_terms.items():
        n = len(v) # number of met genes
        x = len(df_genes.intersection(v))
        pct_overlap = x/(N+n)

        if pct_overlap > 0:
            p_val = hypergeometric_test(total_genes_expressed=M, n_genes_of_interest=n, 
                                        n_genes_picked=N, n_overlap=x)
            hypergeom_res_th_1[key][k] = p_val
        else:
            hypergeom_res_th_1[key][k] = 1

pvn: 2443
nvp: 2476


In [193]:
hypergeom_df_th = pd.DataFrame.from_dict(hypergeom_res_th_1)
hypergeom_df_th['term'] = hypergeom_df_th.index
hypergeom_res_th_1hypergeom_df_th = hypergeom_df_th.melt('term')
hypergeom_df_th['padj_pvn'] = adjust_p_value_fdr(hypergeom_df_th['pvn'])
hypergeom_df_th['padj_nvp'] = adjust_p_value_fdr(hypergeom_df_th['nvp'])
hypergeom_df_th

,pvn,nvp,term,padj_pvn,padj_nvp
KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS,0.801830,1.474229e-03,KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS,1.000000,1.491371e-03
KEGG_MM_CITRATE_CYCLE,0.344419,1.263705e-02,KEGG_MM_CITRATE_CYCLE,0.379297,1.340760e-02
KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY,0.474840,2.091902e-02,KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY,1.000000,4.232452e-02
KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS,0.368829,9.865543e-01,KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS,1.000000,1.000000e+00
KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM,0.863860,1.774566e-03,KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM,1.000000,1.860087e-03
...,...,...,...,...,...
KEGG_MM_MTOR_SIGNALING_PATHWAY,0.444835,6.881465e-04,KEGG_MM_MTOR_SIGNALING_PATHWAY,1.000000,1.663021e-03
KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,0.208335,2.414569e-02,KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,0.604171,6.178455e-02
KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,0.017587,2.644164e-01,KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,0.021858,3.382974e-01
KEGG_MM_INSULIN_SIGNALING_PATHWAY,0.775930,2.273838e-07,KEGG_MM_INSULIN_SIGNALING_PATHWAY,1.000000,2.867013e-07


In [198]:
print(f"pvn: {len(hypergeom_df_th[hypergeom_df_th['padj_pvn'] < 0.05])}\nnvp: {len(hypergeom_df_th[hypergeom_df_th['padj_nvp'] < 0.05])}")
      

pvn: 13
nvp: 17


In [227]:
th_results_pvn = hypergeom_df_th[hypergeom_df_th['padj_pvn'] < 0.05].sort_values('padj_pvn')
th_results_pvn

,pvn,nvp,term,padj_pvn,padj_nvp
KEGG_MM_PYRIMIDINE_METABOLISM,1.643372e-08,0.533050,KEGG_MM_PYRIMIDINE_METABOLISM,2.917824e-08,1.000000
KEGG_MM_METABOLIC_PATHWAYS,8.022960e-08,0.000201,KEGG_MM_METABOLIC_PATHWAYS,2.684606e-07,0.001456
KEGG_MM_PURINE_METABOLISM,2.730702e-05,0.286638,KEGG_MM_PURINE_METABOLISM,1.131291e-04,0.415625
KEGG_MM_CYSTEINE_AND_METHIONINE_METABOLISM,3.280389e-04,0.501645,KEGG_MM_CYSTEINE_AND_METHIONINE_METABOLISM,5.488344e-04,0.765669
KEGG_MM_N-GLYCAN_BIOSYNTHESIS,1.402417e-03,0.384387,KEGG_MM_N-GLYCAN_BIOSYNTHESIS,2.178756e-03,0.903829
KEGG_MM_CYANOAMINO_ACID_METABOLISM,2.332703e-03,1.000000,KEGG_MM_CYANOAMINO_ACID_METABOLISM,5.798432e-03,1.000000
KEGG_MM_AMINOACYL-TRNA_BIOSYNTHESIS,1.496834e-02,0.349687,KEGG_MM_AMINOACYL-TRNA_BIOSYNTHESIS,1.691228e-02,1.000000
KEGG_MM_VALINE_LEUCINE_AND_ISOLEUCINE_BIOSYNTHESIS,1.376746e-02,1.000000,KEGG_MM_VALINE_LEUCINE_AND_ISOLEUCINE_BIOSYNTH...,1.761425e-02,1.000000
KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,1.758727e-02,0.264416,KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,2.185846e-02,0.338297
KEGG_MM_GLYCINE_SERINE_AND_THREONINE_METABOLISM,1.256471e-02,0.858608,KEGG_MM_GLYCINE_SERINE_AND_THREONINE_METABOLISM,2.325808e-02,1.000000


In [228]:
th_results_nvp = hypergeom_df_th[hypergeom_df_th['padj_nvp'] < 0.05].sort_values('padj_nvp')
th_results_nvp

,pvn,nvp,term,padj_pvn,padj_nvp
KEGG_MM_INSULIN_SIGNALING_PATHWAY,7.759302e-01,2.273838e-07,KEGG_MM_INSULIN_SIGNALING_PATHWAY,1.000000e+00,2.867013e-07
KEGG_MM_HIF-1_SIGNALING_PATHWAY,4.574004e-01,5.391479e-06,KEGG_MM_HIF-1_SIGNALING_PATHWAY,5.305844e-01,2.132085e-05
KEGG_MM_AMINO_SUGAR_AND_NUCLEOTIDE_SUGAR_METABOLISM,2.115163e-01,1.079848e-04,KEGG_MM_AMINO_SUGAR_AND_NUCLEOTIDE_SUGAR_METAB...,2.520810e-01,1.105256e-04
KEGG_MM_METABOLIC_PATHWAYS,8.022960e-08,2.008439e-04,KEGG_MM_METABOLIC_PATHWAYS,2.684606e-07,1.456119e-03
KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS,8.018295e-01,1.474229e-03,KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS,1.000000e+00,1.491371e-03
KEGG_MM_MTOR_SIGNALING_PATHWAY,4.448346e-01,6.881465e-04,KEGG_MM_MTOR_SIGNALING_PATHWAY,1.000000e+00,1.663021e-03
KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM,8.638604e-01,1.774566e-03,KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM,1.000000e+00,1.860087e-03
KEGG_MM_INOSITOL_PHOSPHATE_METABOLISM,5.845095e-01,1.635445e-03,KEGG_MM_INOSITOL_PHOSPHATE_METABOLISM,1.000000e+00,1.872154e-03
KEGG_MM_GALACTOSE_METABOLISM,8.933279e-01,2.319721e-03,KEGG_MM_GALACTOSE_METABOLISM,1.000000e+00,2.319721e-03
KEGG_MM_BIOSYNTHESIS_OF_UNSATURATED_FATTY_ACIDS,3.366853e-01,6.619803e-03,KEGG_MM_BIOSYNTHESIS_OF_UNSATURATED_FATTY_ACIDS,5.526722e-01,7.998929e-03


In [229]:
data_dir

'./data/'

In [230]:
th_results_pvn.to_csv('./hypergeom_th_results_Th17p_v_Th17n.csv')

In [231]:
th_results_nvp.to_csv('./hypergeom_th_results_Th17n_v_Th17p.csv')

# apply mann-whitney for DE analysis
> didn't find any genes

In [4]:
compass_data = pd.read_csv(data_dir + 'compass_expression.tsv', sep = '\t') # this is library size normalized already

In [5]:
adata = sc.read_csv(data_dir + 'compass_expression.tsv', delimiter = '\t')
adata = adata.transpose()
adata.obs['sample'] = adata.obs.index
adata

AnnData object with n_obs × n_vars = 40 × 40152
    obs: 'sample'

In [6]:
adata

AnnData object with n_obs × n_vars = 40 × 40152
    obs: 'sample'

In [8]:
adata.obs

,sample
WT-d_S141_L007_R1_001,WT-d_S141_L007_R1_001
Ob-DHA-e_S154_L007_R1_001,Ob-DHA-e_S154_L007_R1_001
WT-1012-c_S135_L007_R1_001,WT-1012-c_S135_L007_R1_001
Ob-DHA-b_S132_L007_R1_001,Ob-DHA-b_S132_L007_R1_001
Ob-EPA-a_S122_L007_R1_001,Ob-EPA-a_S122_L007_R1_001
Ob-c_S136_L007_R1_001,Ob-c_S136_L007_R1_001
Ob-1012-c_S140_L007_R1_001,Ob-1012-c_S140_L007_R1_001
WT-EPA-e_S151_L007_R1_001,WT-EPA-e_S151_L007_R1_001
WT-DHA-d_S143_L007_R1_001,WT-DHA-d_S143_L007_R1_001
WT-DHA-b_S128_L007_R1_001,WT-DHA-b_S128_L007_R1_001


**differential gene expression**

In [45]:
# import random
# adata.obs['louvain'] = ['clust_' + str(random.sample([1,2,3], 1)[0]) for i in range(len(adata.obs))]
# adata.obs['louvain'] = adata.obs['louvain'].astype('category')
# adata.layers['raw'] = adata.X
# adata.layers['scT'] = adata.X

In [106]:
# write simple DE test based off scRNA-methods
# base this off of previous differential expression script
# can't use DeSEq2 because input requires raw counts
def get_count_data(adata, clustering_name = 'louvain', cluster1 = '0', layer = 'lib_norm'):
    cluster1 = [cluster1]
    cluster2 = [str(c) for c in adata.obs[clustering_name].unique()]
    x = adata.layers[layer][adata.obs[clustering_name].isin(cluster1)]
    y = adata.layers[layer][adata.obs[clustering_name].isin(cluster2)]
    return x,y

def get_de_genes(adata, clustering_name = 'sample', cluster = '0', log2FC_threshold = 0.5, padj_threshold = 0.05, layer = 'lib_norm'):
    x,y = get_count_data(adata, clustering_name = clustering_name, cluster1 = cluster, layer = layer)
    corrector_value = np.median(np.squeeze(adata.layers[layer].mean(axis = 0)))
    
    x_mean = np.squeeze(np.asarray((np.sum(x, 0) / x.shape[0])))
    y_mean = np.squeeze(np.asarray((np.sum(y, 0) / y.shape[0])))
    
    log2FC = np.log2((x_mean + corrector_value) / (y_mean + corrector_value))
    log2FC_high = np.where(abs(log2FC) > log2FC_threshold)[0]
    
    pvals = [scipy.stats.mannwhitneyu(x[:, i], y[:, i])[1] for i in log2FC_high]
    pvals_corrected = [p * len(log2FC_high) for p in pvals]
    
    log2FC_data = {'log2FC': log2FC[log2FC_high].tolist(),
                   'x_mean': x_mean[log2FC_high], 'y_mean': y_mean[log2FC_high],
                   'pval': pvals, 'padj': pvals_corrected}
    
    log2FC_data = pd.DataFrame(log2FC_data, index=adata.var_names[log2FC_high])
    log2FC_data = log2FC_data[log2FC_data['padj'] < padj_threshold].sort_values(by='log2FC', ascending=False)
    
    return(log2FC_data)

def get_de_genes_each_vs_all(adata, clustering_name = 'sample', log2FC_threshold = 0.5, padj_threshold = 0.05, layer = 'lib_norm'):
    all_logFC_data = {}
    for cluster in adata.obs[clustering_name].unique():
        print(cluster)
        all_logFC_data[cluster] = get_de_genes(adata, clustering_name = clustering_name, cluster = cluster, log2FC_threshold = log2FC_threshold, 
                                             padj_threshold = padj_threshold, layer = layer)
        adata.uns['%s_logFC' % (clustering_name)] = all_logFC_data
    return all_logFC_data

In [75]:
de_logFC_thresh = 0.5
de_p_thresh = 0.01

In [84]:
adata.layers['lib_norm'] = adata.X

In [107]:
de_genes = get_de_genes_each_vs_all(adata, 'sample', log2FC_threshold=de_logFC_thresh, padj_threshold=de_p_thresh)

WT-d_S141_L007_R1_001
Ob-DHA-e_S154_L007_R1_001
WT-1012-c_S135_L007_R1_001
Ob-DHA-b_S132_L007_R1_001
Ob-EPA-a_S122_L007_R1_001
Ob-c_S136_L007_R1_001
Ob-1012-c_S140_L007_R1_001
WT-EPA-e_S151_L007_R1_001
WT-DHA-d_S143_L007_R1_001
WT-DHA-b_S128_L007_R1_001
WT-EPA-a_S117_L007_R1_001
WT-1012-d_S145_L007_R1_001
Ob-1012-d_S149_L007_R1_001
WT-DHA-a_S118_L007_R1_001
Ob-911-c_S139_L007_R1_001
Ob-d_S146_L007_R1_001
WT-EPA-b_S127_L007_R1_001
Ob-911-a_S124_L007_R1_001
Ob-b_S131_L007_R1_001
WT-1012-b_S130_L007_R1_001
Ob-EPA-c_S137_L007_R1_001
Ob-EPA-e_S153_L007_R1_001
Ob-1012-b_S133_L007_R1_001
WT-911-b_S129_L007_R1_001
WT-911-a_S119_L007_R1_001
WT-DHA-c_S134_L007_R1_001
WT-EPA-d_S142_L007_R1_001
Ob-a_S121_L007_R1_001
Ob-DHA-c_S138_L007_R1_001
WT-1012-a_S120_L007_R1_001
Ob-1012-a_S125_L007_R1_001
Ob-DHA-a_S123_L007_R1_001
WT-e_S150_L007_R1_001
Ob-911-e_S155_L007_R1_001
WT-a_S116_L007_R1_001
Ob-911-d_S148_L007_R1_001
WT-1012-e_S152_L007_R1_001
WT-911-d_S144_L007_R1_001
Ob-DHA-d_S147_L007_R1_001
WT-b_

In [109]:
de_genes

{'WT-d_S141_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-DHA-e_S154_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'WT-1012-c_S135_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-DHA-b_S132_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-EPA-a_S122_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-c_S136_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-1012-c_S140_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'WT-EPA-e_S151_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'WT-DHA-d_S143_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'WT-DHA-b_S128_L007_R1_001': Empty DataFrame
 Columns: [log2

In [10]:
t = pd.read_csv('/Genomics/pritykinlab/amanda/other/metabolic_analysis/metabolic_analysis/data/th_data/GSE162300_DFMO_RNA_est_counts.csv')

In [11]:
t

,symbol,Th17p_Vehicle_WT1_run1,Th17p_Vehicle_WT1_run2,Th17n_Vehicle_WT1_run1,Th17n_Vehicle_WT1_run2,iTreg_Vehicle_WT1_run1,iTreg_Vehicle_WT1_run2,Th17p_DFMO_WT1_run1,Th17p_DFMO_WT1_run2,Th17n_DFMO_WT1_run1,...,Th17n_Vehicle_WT3_run1,Th17n_Vehicle_WT3_run2,iTreg_Vehicle_WT3_run1,iTreg_Vehicle_WT3_run2,Th17p_DFMO_WT3_run1,Th17p_DFMO_WT3_run2,Th17n_DFMO_WT3_run1,Th17n_DFMO_WT3_run2,iTreg_DFMO_WT3_run1,iTreg_DFMO_WT3_run2
0,0610007P14RIK,787.00,870.00,708.00,845.00,570.00,624.00,786.0,846.00,864.00,...,1183.00,1332.00,794.00,853.00,928.00,1045.00,1120.00,1163.00,710.00,723.00
1,0610009B22RIK,93.00,98.00,66.00,70.00,87.90,100.00,123.0,128.00,106.00,...,94.00,76.00,109.00,145.00,121.00,131.00,131.00,137.78,137.00,148.00
2,0610009L18RIK,2.00,2.00,2.00,1.00,3.00,4.00,4.0,5.00,3.00,...,6.00,3.00,0.00,0.00,5.00,11.00,3.00,4.00,4.00,5.00
3,0610009O20RIK,177.00,206.00,148.00,170.00,194.00,159.00,168.0,172.00,161.00,...,249.00,284.00,202.00,254.98,216.00,228.98,212.00,226.00,258.97,246.00
4,0610010F05RIK,24.00,23.00,23.00,25.00,41.00,43.00,32.0,22.00,32.00,...,46.00,39.00,30.00,40.00,44.00,55.00,24.00,31.00,20.00,14.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20812,ZYG11A,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
20813,ZYG11B,302.34,291.59,215.25,245.82,229.23,251.62,169.6,210.45,180.24,...,359.07,375.34,386.87,405.60,242.78,271.74,276.50,265.54,271.90,273.43
20814,ZYX,250.00,320.00,328.00,422.00,177.00,226.00,288.0,331.00,378.00,...,586.00,627.00,279.00,290.00,420.00,484.00,533.00,629.00,200.00,232.00
20815,ZZEF1,220.00,230.00,145.00,177.00,235.00,238.00,154.0,199.00,175.00,...,203.00,261.00,258.00,283.00,243.00,262.00,275.00,274.00,325.00,316.00


In [14]:
t = pd.read_csv('/Genomics/pritykinlab/amanda/other/metabolic_analysis/metabolic_analysis/data/th_data/74833_samps/GSM1943520_single-cells_Tgfb1_IL6-IL17A_POS_48hr_140519_T16_IL17A_48h_SC90_408.genes.fpkm_tracking', sep = '\t')

In [17]:
t['FPKM'].unique()

array([   0.    ,   25.5533,  223.804 , ...,   84.2759,   24.3575,
       4556.79  ])